# 智慧運算技術導論 (Week 4 自然語言處理主題 - Day3)
今日教學內容：
- 白話文衛教翻譯機應用：將艱澀的病歷摘要或處方說明，轉換為阿公阿嬤都聽得懂的衛教資訊。
- 本教學不需使用本地或 Colab GPU。
- 示範使用 [Groq](https://console.groq.com/home) 來互叫 [gpt-oss-120b](https://openai.com/zh-Hant/index/introducing-gpt-oss/)。
- LLM 框架：[aisuite](https://github.com/andrewyng/aisuite)；視覺化框架：[Gradio](https://www.gradio.app/guides/quickstart)

In [1]:
# 0. 安裝必要套件

!pip install 'aisuite[all]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of mistralai to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.9 MB/s eta 0:00:00
INFO: pip is still looking at multiple versions of mistralai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 

In [1]:
# 1. 引入套件

import os
import gradio as gr
from google.colab import userdata
import aisuite as ai
client = ai.Client()

In [2]:
# 2. 設定 API key
# API: Application Programming Interface

os.environ['GROQ_API_KEY'] = "YOUR_API_KEY"
# 或是使用 Colab Secret Key 方式
# os.environ['GROQ_API_KEY'] = userdata.get('groq')

In [3]:
# 3. 設定要使用的模型
# 更多模型請見 https://console.groq.com/docs/models

model = "groq:openai/gpt-oss-120b"

In [4]:
# 4. LLM 出任務
# 以講笑話為範例

messages = [
    {"role": "system", "content": "請用台灣習慣的中文回覆。"}, # system prompt
    {"role": "user", "content": "跟我說一個笑話吧！"}, # user prompt
]

response = client.chat.completions.create(
    model=model,
    messages=messages,
)
print(response.choices[0].message.content)

哈哈，來個輕鬆的笑話吧：

有一天，小明跟老師說：「老師，我昨天晚上在做夢，夢到自己變成一顆水果。」

老師好奇地問：「哦？變成什麼水果呀？」

小明回答：「變成『蘋果』，因為我在夢裡一直在『蘋果派』（平）！」

老師哭笑不得，說：「小明，你這個笑話真是太'果'斷了！」 😄

（註：「蘋果派」跟「平」諧音，讓人想起「平安」的祝福，兼顧了台灣人常吃蘋果派的小情懷。）祝你今天開心滿分，笑口常開！


In [6]:
# 5. 設定提示詞 (Prompts)

def generate_text(user_input: str) -> str:
    prompt = f"""
你的任務是將輸入的「專業醫療術語」或「病歷摘要」，翻譯成台灣民眾容易理解的「白話文衛教建議」。
請注意：
1. 語氣要溫柔、正面。
2. 避免使用任何英文專有名詞，若必須使用請加上解釋。
3. 條列式重點，讓長輩容易閱讀。
使用者的輸入如下：
{user_input}
"""

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "你是一位親切、專業的資深藥師與衛教人員。"},
            {"role": "user", "content": prompt},
        ],
        max_tokens=800,
    )
    return response.choices[0].message.content

In [9]:
# 6. 透過 Gradio 建立介面

with gr.Blocks(theme=gr.themes.Base()) as demo:
    gr.Markdown("## 🏥 醫療術語 -> 白話文衛教翻譯機")
    gr.Markdown("輸入醫師的病歷摘要或藥名，AI 幫你轉換成給病人看的衛教單。")

    input_text = gr.Textbox(
        label="病歷摘要或藥名",
        placeholder="例如：Pt presents with GERD symptoms. Prescribed PPI q.d. advice lifestyle modification.",
    )
    generate_btn = gr.Button("生成衛教單")
    output_text = gr.Markdown(label="病人衛教資訊")

    # 範例按鈕 (可以不用自己打字)
    gr.Examples(
        examples=[
            ["Diagnosis: Type 2 DM. Current HbA1c 8.5%. Start Metformin 500mg b.i.d. Watch for GI upset."],
            ["Patient has acute pharyngitis. S/S: sore throat, fever. Rx: Acetaminophen 500mg q4h prn."],
        ],
        inputs=input_text
    )

    generate_btn.click(fn=generate_text, inputs=input_text, outputs=output_text)

/tmp/ipython-input-1189745952.py:2: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Base()) as demo:


In [10]:
# 7. 執行 Gradio 介面

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://3c3398726ee86514a4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://3c3398726ee86514a4.gradio.live
